## Movie Data

In [5]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [6]:
!ls ../zippedData/

bom.movie_gross.csv.gz
im.db
im.db.zip
im2.db
rt.movie_info.tsv.gz
rt.reviews.tsv.gz
tmdb.movies.csv.gz
tn.movie_budgets.csv.gz


In [7]:
conn = sqlite3.connect('../zippedData/im2.db')

In [8]:
pd.read_sql("SELECT * FROM sqlite_master", conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


# QUESTIONS
- What are the causation between movie budget and total gross?
    - is foreign higher than domestic or vice versa?
- What are the relevant markers between movie budget and movie ratings?
    - advertisement foreign and domestic?
- What genres have grossed the most revenue?

# IMDB MOVIE BASICS

In [9]:
imdb_basics = pd.read_sql("SELECT * FROM movie_basics", conn)
imdb_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


# IMDB RATINGS

In [10]:
imdb_ratings = pd.read_sql("SELECT * FROM movie_ratings", conn)
imdb_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


# MERGE OF TOP TWO TABLES: basic_ratings
-merging imdb_ratings and imdb_basics allows for movie ratings to with movie name, genres, and release year. This allows the data to be easily read. 

In [12]:
basic_ratings = pd.merge(imdb_basics, imdb_ratings, on = "movie_id", how = "left")

In [13]:
basic_ratings

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0
...,...,...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama,NaN,NaN
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary,NaN,NaN
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy,NaN,NaN
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None,NaN,NaN


In [31]:
basic_ratings['movie_year'] = basic_ratings["primary_title"] + " " + basic_ratings['start_year'].astype('str')
#allows for a unique identifier to merge on across dataframes not in the same network ("movie_year").

In [23]:
basic_ratings

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,movie_year
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Sunghursh 2013
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,One Day Before the Rainy Season 2019
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,The Other Side of the Wind 2018
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,Sabse Bada Sukh 2018
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,The Wandering Soap Opera 2017
...,...,...,...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama,NaN,NaN,Kuambil Lagi Hatiku 2019
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary,NaN,NaN,Rodolpho Teóphilo - O Legado de um Pioneiro 2015
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy,NaN,NaN,Dankyavar Danka 2013
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None,NaN,NaN,6 Gunn 2017


## MOVIE GROSS 

In [34]:
movie_gross = pd.read_csv("../zippedData/bom.movie_gross.csv.gz")
movie_gross.head(2)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


In [35]:
 movie_gross['domestic_gross'] = movie_gross['domestic_gross'].fillna(0)
# I am filling NULL with zeros because it alleviate messy data and allow an accurate number to be found for total_gross.

In [36]:
movie_gross['foreign_gross'] = movie_gross['foreign_gross'].str.replace(",", "")
# the foreign_gross column had a value that contained a comma ex. 1,000. This comma did not allow foreign and domestic
# gross to be added together because foreign Dtype was an object instead of a float or integer. 

In [37]:
 movie_gross['foreign_gross'] = movie_gross['foreign_gross'].fillna(0)
# I am filling NULL with zeros because it alleviate messy data and allow an accurate number to be found for total_gross.

In [38]:
movie_gross['foreign_gross'] = movie_gross['foreign_gross'].astype(float)
# this will turn the foreign_gross column into a float and allow it to be added to the domestic_gross 
# and get a tot_gross column.

In [39]:
movie_gross['tot_gross'] = movie_gross['foreign_gross'] + movie_gross['domestic_gross']

In [40]:
movie_gross['movie_year'] = movie_gross["title"] + " " + movie_gross['year'].astype('str')
#allows for a unique identifier to merge on across dataframes not in the same network ("movie_year").

In [41]:
movie_gross

,title,studio,domestic_gross,foreign_gross,year,tot_gross,movie_year
0,Toy Story 3,BV,415000000.0,652000000.0,2010,1.067000e+09,Toy Story 3 2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,1.025500e+09,Alice in Wonderland (2010) 2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,9.603000e+08,Harry Potter and the Deathly Hallows Part 1 2010
3,Inception,WB,292600000.0,535700000.0,2010,8.283000e+08,Inception 2010
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,7.526000e+08,Shrek Forever After 2010
...,...,...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,0.0,2018,6.200000e+03,The Quake 2018
3383,Edward II (2018 re-release),FM,4800.0,0.0,2018,4.800000e+03,Edward II (2018 re-release) 2018
3384,El Pacto,Sony,2500.0,0.0,2018,2.500000e+03,El Pacto 2018
3385,The Swan,Synergetic,2400.0,0.0,2018,2.400000e+03,The Swan 2018


# MOVIE BUDGETS

In [46]:
budg_mov = pd.read_csv("../zippedData/tn.movie_budgets.csv.gz")

In [47]:
budg_mov.head(10)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,053,311,220"
6,7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200"
7,8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,"$300,000,000","$309,420,425","$963,420,425"
8,9,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209"
9,10,"Nov 6, 2015",Spectre,"$300,000,000","$200,074,175","$879,620,923"


In [48]:
a = 'Dec 18, 2009'

In [62]:
budg_mov['movie'] + " " + budg_mov['release_date'].str[-4:]
# this allows for the year to pulled from the release_date and adding it to the movie column that 

0                                            Avatar 2009
1       Pirates of the Caribbean: On Stranger Tides 2011
2                                      Dark Phoenix 2019
3                           Avengers: Age of Ultron 2015
4                 Star Wars Ep. VIII: The Last Jedi 2017
                              ...                       
5777                                         Red 11 2018
5778                                      Following 1999
5779                  Return to the Land of Wonders 2005
5780                           A Plague So Pleasant 2015
5781                              My Date With Drew 2005
Length: 5782, dtype: object

In [60]:
budg_mov['movie_year'] = budg_mov['movie'] + " " + budg_mov['release_date'].str[-4:]

In [61]:
budg_mov

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,movie_year
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",Avatar 2009
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",Pirates of the Caribbean: On Stranger Tides 2011
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",Dark Phoenix 2019
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",Avengers: Age of Ultron 2015
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",Star Wars Ep. VIII: The Last Jedi 2017
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0,Red 11 2018
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495",Following 1999
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338",Return to the Land of Wonders 2005
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0,A Plague So Pleasant 2015


# Merge of Messiness

In [42]:
df = basic_ratings.merge(movie_gross, on='movie_year')
# main source of data has "movie_gross", "imdb_ratings", "imdb_basics"(merged into the basic_ratings) 
# merged into one table to create df. This will allow for allow the neccessary data to be pulled from one source and better
# organized.

In [63]:
df = df.merge(budg_mov, on='movie_year')

In [66]:
df.head(10)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,movie_year,title,...,domestic_gross_x,foreign_gross,year,tot_gross,id,release_date,movie,production_budget,domestic_gross_y,worldwide_gross
0,tt0359950,The Secret Life of Walter Mitty,The Secret Life of Walter Mitty,2013,114.0,"Adventure,Comedy,Drama",7.3,275300.0,The Secret Life of Walter Mitty 2013,The Secret Life of Walter Mitty,...,58200000.0,129900000.0,2013,188100000.0,37,"Dec 25, 2013",The Secret Life of Walter Mitty,"$91,000,000","$58,236,838","$187,861,183"
1,tt0365907,A Walk Among the Tombstones,A Walk Among the Tombstones,2014,114.0,"Action,Crime,Drama",6.5,105116.0,A Walk Among the Tombstones 2014,A Walk Among the Tombstones,...,26300000.0,26900000.0,2014,53200000.0,67,"Sep 19, 2014",A Walk Among the Tombstones,"$28,000,000","$26,017,685","$62,108,587"
2,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",7.0,539338.0,Jurassic World 2015,Jurassic World,...,652300000.0,1019.4,2015,652301019.4,34,"Jun 12, 2015",Jurassic World,"$215,000,000","$652,270,625","$1,648,854,864"
3,tt0376136,The Rum Diary,The Rum Diary,2011,119.0,"Comedy,Drama",6.2,94787.0,The Rum Diary 2011,The Rum Diary,...,13100000.0,10800000.0,2011,23900000.0,16,"Oct 28, 2011",The Rum Diary,"$45,000,000","$13,109,815","$21,544,732"
4,tt0383010,The Three Stooges,The Three Stooges,2012,92.0,"Comedy,Family",5.1,28570.0,The Three Stooges 2012,The Three Stooges,...,44300000.0,10500000.0,2012,54800000.0,4,"Apr 13, 2012",The Three Stooges,"$30,000,000","$44,338,224","$54,052,249"
5,tt0398286,Tangled,Tangled,2010,100.0,"Adventure,Animation,Comedy",7.8,366366.0,Tangled 2010,Tangled,...,200800000.0,391000000.0,2010,591800000.0,15,"Nov 24, 2010",Tangled,"$260,000,000","$200,821,936","$586,477,240"
6,tt0401729,John Carter,John Carter,2012,132.0,"Action,Adventure,Sci-Fi",6.6,241792.0,John Carter 2012,John Carter,...,73100000.0,211100000.0,2012,284200000.0,14,"Mar 9, 2012",John Carter,"$275,000,000","$73,058,679","$282,778,100"
7,tt0427152,Dinner for Schmucks,Dinner for Schmucks,2010,114.0,Comedy,5.9,91546.0,Dinner for Schmucks 2010,Dinner for Schmucks,...,73000000.0,13800000.0,2010,86800000.0,51,"Jul 30, 2010",Dinner for Schmucks,"$69,000,000","$73,026,337","$86,796,502"
8,tt0429493,The A-Team,The A-Team,2010,117.0,"Action,Adventure,Thriller",6.8,235256.0,The A-Team 2010,The A-Team,...,77200000.0,100000000.0,2010,177200000.0,32,"Jun 11, 2010",The A-Team,"$110,000,000","$77,222,099","$177,241,171"
9,tt0431021,The Possession,The Possession,2012,92.0,"Horror,Mystery,Thriller",5.9,53649.0,The Possession 2012,The Possession,...,49100000.0,36300000.0,2012,85400000.0,81,"Aug 31, 2012",The Possession,"$14,000,000","$49,130,588","$82,925,064"


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1052 entries, 0 to 1051
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie_id           1052 non-null   object 
 1   primary_title      1052 non-null   object 
 2   original_title     1052 non-null   object 
 3   start_year         1052 non-null   int64  
 4   runtime_minutes    1047 non-null   float64
 5   genres             1051 non-null   object 
 6   averagerating      1043 non-null   float64
 7   numvotes           1043 non-null   float64
 8   movie_year         1052 non-null   object 
 9   title              1052 non-null   object 
 10  studio             1052 non-null   object 
 11  domestic_gross_x   1052 non-null   float64
 12  foreign_gross      1052 non-null   float64
 13  year               1052 non-null   int64  
 14  tot_gross          1052 non-null   float64
 15  id                 1052 non-null   int64  
 16  release_date       1052 

# Question 1:
- What are the causation between movie budget and total gross?
    - is foreign higher than domestic or vice versa?

# Question 2:
- What are the relevant markers between movie budget and movie ratings?
    - advertisement foreign and domestic?


# Question 3:
- What genres have grossed the most revenue?

In [16]:
t_numvotes = df.sort_values('numvotes', ascending=False)[:25]
# this rated top 25 by number of votes

In [19]:
t_numvotes = t_numvotes.sort_values('averagerating', ascending=False)
# this rated top 25 but ordered them by ascending order of average ratings. 

In [21]:
t_numvotes.head(10)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
7066,tt1375666,Inception,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,1841066.0
311,tt0816692,Interstellar,Interstellar,2014,169.0,"Adventure,Drama,Sci-Fi",8.6,1299334.0
72820,tt4154756,Avengers: Infinity War,Avengers: Infinity War,2018,149.0,"Action,Adventure,Sci-Fi",8.5,670926.0
13044,tt1675434,The Intouchables,Intouchables,2011,112.0,"Biography,Comedy,Drama",8.5,677343.0
20342,tt1853728,Django Unchained,Django Unchained,2012,165.0,"Drama,Western",8.4,1211405.0
6900,tt1345836,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769.0
104,tt0435761,Toy Story 3,Toy Story 3,2010,103.0,"Adventure,Animation,Comedy",8.3,682218.0
545,tt0993846,The Wolf of Wall Street,The Wolf of Wall Street,2013,180.0,"Biography,Crime,Drama",8.2,1035358.0
25432,tt2015381,Guardians of the Galaxy,Guardians of the Galaxy,2014,121.0,"Action,Adventure,Comedy",8.1,948394.0
356,tt0848228,The Avengers,The Avengers,2012,143.0,"Action,Adventure,Sci-Fi",8.1,1183655.0


In [10]:
movie_gross[movie_gross['title'] == 'Avatar']
#how to find the a specific movie


,title,studio,domestic_gross,foreign_gross,year


In [11]:
type(movie_gross)

pandas.core.frame.DataFrame

In [12]:
movie_gross.index

RangeIndex(start=0, stop=3387, step=1)

In [13]:
movie_gross['foreign_gross'].sort_values(ascending=False)

1331      99700
1805      99700
302       99600
510     9900000
3181    9900000
         ...   
3382        NaN
3383        NaN
3384        NaN
3385        NaN
3386        NaN
Name: foreign_gross, Length: 3387, dtype: object

In [14]:
movie_gross.columns

Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')

In [15]:
movie_gross.values

array([['Toy Story 3', 'BV', 415000000.0, '652000000', 2010],
       ['Alice in Wonderland (2010)', 'BV', 334200000.0, '691300000',
        2010],
       ['Harry Potter and the Deathly Hallows Part 1', 'WB', 296000000.0,
        '664300000', 2010],
       ...,
       ['El Pacto', 'Sony', 2500.0, nan, 2018],
       ['The Swan', 'Synergetic', 2400.0, nan, 2018],
       ['An Actor Prepares', 'Grav.', 1700.0, nan, 2018]], dtype=object)

In [16]:
movie_gross.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


In [17]:
movie_gross.shape

(3387, 5)

In [18]:
movie_gross.mean()

domestic_gross    2.874585e+07
year              2.013958e+03
dtype: float64

In [19]:
movie_gross.min()

title              '71
domestic_gross     100
year              2010
dtype: object

In [20]:
movie_gross.max()

title             xXx: The Return of Xander Cage
domestic_gross                         9.367e+08
year                                        2018
dtype: object

In [22]:
movie_gross.describe(include=[object])

,title,studio,foreign_gross
count,3387,3382,2037
unique,3386,257,1204
top,Bluebeard,IFC,1200000
freq,2,166,23


In [23]:
movie_gross.head(1)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010


In [56]:
movie_gross['domestic_gross'].value_counts()

1100000.0      32
1000000.0      30
1300000.0      30
1200000.0      25
1400000.0      23
               ..
68800.0         1
87000000.0      1
739000.0        1
336000000.0     1
727000.0        1
Name: domestic_gross, Length: 1797, dtype: int64

In [72]:
movie_gross['foreign_gross'].value_counts()

0            1350
1200000        23
1100000        14
1900000        12
4200000        12
             ... 
211400000       1
244800000       1
140500000       1
409300000       1
33000000        1
Name: foreign_gross, Length: 1205, dtype: int64

In [25]:
movie_gross['domestic_gross'] >= 500000000

0       False
1       False
2       False
3       False
4       False
        ...  
3382    False
3383    False
3384    False
3385    False
3386    False
Name: domestic_gross, Length: 3387, dtype: bool

In [86]:
sum(movie_gross['domestic_gross'] >= 500000000)
# this shows that 9 movies grossed over 500M domestically. This is important because it will show the popularity of the
# movie within the original country of release.

9

In [84]:
sum(movie_gross['foreign_gross'] >= 500000000)
# this shows that 58 movies grossed over 500M foreign. This is important because it will show the popularity of the 
# movie within the foreign countries and will allow one to see if the 9 domestic movies were received worldwide.

58

In [85]:
sum(movie_gross['tot_gross'] >= 500000000)
# this shows that over half of the movies grossed lessed than 500M in for the foreign or domestic category but combined
# they meet the threshold. 

126

In [87]:
big_bucks = movie_gross.loc[movie_gross['tot_gross'] >= 500000000]

In [88]:
big_bucks

,title,studio,domestic_gross,foreign_gross,year,tot_gross
0,Toy Story 3,BV,415000000.0,652000000.0,2010,1.067000e+09
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,1.025500e+09
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,9.603000e+08
3,Inception,WB,292600000.0,535700000.0,2010,8.283000e+08
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,7.526000e+08
...,...,...,...,...,...,...
3092,Detective Chinatown 2,WB,2000000.0,542100000.0,2018,5.441000e+08
3093,The Meg,WB,145400000.0,384800000.0,2018,5.302000e+08
3094,Ralph Breaks the Internet,BV,201100000.0,328100000.0,2018,5.292000e+08
3095,Hotel Transylvania 3: Summer Vacation,Sony,167500000.0,361100000.0,2018,5.286000e+08


In [65]:
movie_gross['studio'].unique()

array(['BV', 'WB', 'P/DW', 'Sum.', 'Par.', 'Uni.', 'Fox', 'Wein.', 'Sony',
       'FoxS', 'SGem', 'WB (NL)', 'LGF', 'MBox', 'CL', 'W/Dim.', 'CBS',
       'Focus', 'MGM', 'Over.', 'Mira.', 'IFC', 'CJ', 'NM', 'SPC', 'ParV',
       'Gold.', 'JS', 'RAtt.', 'Magn.', 'Free', '3D', 'UTV', 'Rela.',
       'Zeit.', 'Anch.', 'PDA', 'Lorb.', 'App.', 'Drft.', 'Osci.', 'IW',
       'Rog.', nan, 'Eros', 'Relbig.', 'Viv.', 'Hann.', 'Strand', 'NGE',
       'Scre.', 'Kino', 'Abr.', 'CZ', 'ATO', 'First', 'GK', 'FInd.',
       'NFC', 'TFC', 'Pala.', 'Imag.', 'NAV', 'Arth.', 'CLS', 'Mont.',
       'Olive', 'CGld', 'FOAK', 'IVP', 'Yash', 'ICir', 'FM', 'Vita.',
       'WOW', 'Truly', 'Indic.', 'FD', 'Vari.', 'TriS', 'ORF', 'IM',
       'Elev.', 'Cohen', 'NeoC', 'Jan.', 'MNE', 'Trib.', 'Rocket',
       'OMNI/FSR', 'KKM', 'Argo.', 'SMod', 'Libre', 'FRun', 'WHE', 'P4',
       'KC', 'SD', 'AM', 'MPFT', 'Icar.', 'AGF', 'A23', 'Da.', 'NYer',
       'Rialto', 'DF', 'KL', 'ALP', 'LG/S', 'WGUSA', 'MPI', 'RTWC', 'FIP

In [31]:
movie_gross.sort_values(by='domestic_gross')

,title,studio,domestic_gross,foreign_gross,year
1476,Storage 24,Magn.,100.0,NaN,2013
2321,The Chambermaid,FM,300.0,NaN,2015
2756,News From Planet Mars,KL,300.0,NaN,2016
2757,Satanic,Magn.,300.0,NaN,2016
1018,Apartment 143,Magn.,400.0,426000,2012
...,...,...,...,...,...
1975,Surprise - Journey To The West,AR,NaN,49600000,2015
2392,Finding Mr. Right 2,CL,NaN,114700000,2016
2468,Solace,LGP,NaN,22400000,2016
2595,Viral,W/Dim.,NaN,552000,2016


In [29]:
movie_gross['year'].value_counts()


2015    450
2016    436
2012    400
2011    399
2014    395
2013    350
2010    328
2017    321
2018    308
Name: year, dtype: int64

# IMDB DIRECTORS

In [53]:
imdb_direc = pd.read_sql("SELECT * FROM directors", conn)
imdb_direc.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


# IMDB AKAS

In [54]:
imdb_akas = pd.read_sql("SELECT * FROM movie_akas", conn)
imdb_akas.head()

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0


# IMDB KNOWN FOR

In [46]:
imdb_kfor = pd.read_sql("SELECT * FROM known_for ", conn)
imdb_kfor.head()

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534


# IMDB PERSONS

In [48]:
imdb_persons = pd.read_sql("SELECT * FROM persons", conn)
imdb_persons.head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


# IMDB PRINCIPALS

In [49]:
imdb_princ = pd.read_sql("SELECT * FROM principals", conn)
imdb_princ.head()

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"


# IMDB WRITERS

In [50]:
imdb_writers = pd.read_sql("SELECT * FROM writers", conn)
imdb_writers.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087


## TMDB MOVIES

In [33]:
tmdb_mov = pd.read_csv("../zippedData/tmdb.movies.csv.gz")

In [34]:
tmdb_mov.head(1)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788


In [35]:
tmdb_mov.tail(1)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
26516,26516,"[53, 27]",309885,en,The Church,0.6,2018-10-05,The Church,0.0,1


In [39]:
budg_mov.mean()

id    50.372363
dtype: float64

In [40]:
budg_mov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [41]:
budg_mov.describe()

,id
count,5782.000000
mean,50.372363
std,28.821076
min,1.000000
25%,25.000000
50%,50.000000
75%,75.000000
max,100.000000


In [42]:
budg_mov.describe(include=[object])

,release_date,movie,production_budget,domestic_gross,worldwide_gross
count,5782,5782,5782,5782,5782
unique,2418,5698,509,5164,5356
top,"Dec 31, 2014",Halloween,"$20,000,000",$0,$0
freq,24,3,231,548,367


In [43]:
budg_mov.shape

(5782, 6)

In [44]:
budg_mov['domestic_gross']

0       $760,507,625
1       $241,063,875
2        $42,762,350
3       $459,005,868
4       $620,181,382
            ...     
5777              $0
5778         $48,482
5779          $1,338
5780              $0
5781        $181,041
Name: domestic_gross, Length: 5782, dtype: object

In [45]:
budg_mov['worldwide_gross']

0       $2,776,345,279
1       $1,045,663,875
2         $149,762,350
3       $1,403,013,963
4       $1,316,721,747
             ...      
5777                $0
5778          $240,495
5779            $1,338
5780                $0
5781          $181,041
Name: worldwide_gross, Length: 5782, dtype: object